In [82]:
import pandas as pd
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import glob
from tqdm.notebook import tqdm
import os
from sklearn.metrics import f1_score

In [106]:
path = '../ignored/tvsum50_ver_1_1/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
df = pd.read_csv(path,delimiter='\t', header=None, names=['id','cat','scores'])
df = df.join(df['scores'].str.split(',',expand=True).apply(pd.to_numeric))
df = df.drop(axis=1, columns=['scores'])

In [125]:
# files = ['EE-bNr36nyA']
# files = ['98MoyGZKHXc', 'Bhxk-O1Y7Ho', '4wU_LUjG5Ic', 'iVt07TCkFM0']
files = []
for file_id in glob.glob('../inputs/*'):
    filename = file_id.split('/')[-1].split('.')[0]
    if filename not in files:
        files.append(filename)
proportion = 0.2

Get the predicted indices

In [126]:
temp = []
table = []
for file_id in files:
    frame_scores_for_video = df[df['id']==file_id].drop(axis=1, columns=['id','cat']).values
    frame_scores_for_video_nonan = frame_scores_for_video[~np.isnan(frame_scores_for_video)]
    frame_scores_for_video_nonan = frame_scores_for_video_nonan.reshape((20,-1))
    
    with open(f'../intermediate/{file_id}_clip_list.json') as f:
        clip_data = json.load(f)

    # Convert clip start time and end time to fps
    fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
    for clip in clip_data:
        h,m,s = clip['clip_start'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_start_fps'] = round(secs*fps)
        h,m,s = clip['clip_end'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_end_fps'] = round(secs*fps)
        temp.append([clip['clip_start_fps'], clip['clip_end_fps']])
    # Generate list with included and excluded frames. Included frames are 1.
    val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
    try:
        for clip in clip_data:
            for i in range(clip['clip_start_fps'], clip['clip_end_fps']+1):
                val_list[i] = 1
        val_np = np.array(val_list)
    except:
        pass
    avg_frame_scores = frame_scores_for_video_nonan.mean(axis=0)
    fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
    last = 0
    l = []
    for i,el in enumerate(list(frame_scores_for_video_nonan.mean(axis=0))):
        if i % (fps*2) == 0:
            d = {}
            last = i
            d['start'] = i
            d['score'] = el
            l.append(d)

    if last<len(avg_frame_scores):
        l.append({'start': len(avg_frame_scores)-1, 'score': avg_frame_scores[-1]})
    arranged = sorted(l,key=lambda x: x['score'], reverse=True)
    selected_list = [0 for i in range(len(avg_frame_scores))]
    target_length = round((len(avg_frame_scores))*proportion)
    for selected in arranged:
        for i in range(selected['start'],selected['start']+2*fps):
            try:
                selected_list[i] = 1
            except IndexError:
                pass
        target_length-=2*fps
        if target_length<0:
            break
    table.append({"file_id":file_id, "cat": df[df['id']==file_id]['cat'].iloc[0], "f1": f1_score(y_true=selected_list, y_pred=val_list)})
    print(file_id, df[df['id']==file_id]['cat'].iloc[0], f1_score(y_true=selected_list, y_pred=val_list))

vdmoEJ5YbrQ VU 0.319868771785934
HT5vyqe0Xaw VU 0.19548133595284872
eQu1rNs0an0 BT 0.3430777716492003
WxtbjNsCQ8A BK 0.31279342723004694
37rzWOQsNIw MS 0.32
AwmHb44_ouw VT 0.02862595419847328
0tmA_C6XwfM GA 0.26427622841965476
b626MiF1ew4 PK 0.1293219577907499
EYqVtI9YWJA BT 0.42775510204081635
jcoYJXDG9sw DS 0.09090909090909091
Se3oxnaPsz0 BK 0.25810253388332355
qqR6AEXwxoQ BT 0.20888661899897856
Bhxk-O1Y7Ho GA 0.2647367316970841
J0nA4VgnoCo VT 0.15135929463629683
XzYM3PfTM4w VT 0.26704953338119164
E11zDS9XGzg DS 0.234315424610052
3eYKfiOEJNs GA 0.03292568203198495
JgHubY5Vw3Y BT 0.30992196209587514
i3wAGJaaktw GA 0.3673469387755102
WG0MBPpPC6I MS 0.14965259219668625
Yi4Ij2NM7U4 MS 0.29663906478324403
EE-bNr36nyA BK 0.03591836734693878
iVt07TCkFM0 BT 0.46068455134135067
98MoyGZKHXc VT 0.10774410774410775
akI8YFjEmUw VU 0.154320987654321
xwqBXPGE9pQ VU 0.28264758497316633
Hl-__g2gn_A MS 0.1469767441860465
4wU_LUjG5Ic PR 0.20567755757900374


In [131]:
for l in sorted(table, key=lambda x: x['f1'], reverse=True):
    print(l['file_id'], l['cat'], round(l['f1'],5))

iVt07TCkFM0 BT 0.46068
EYqVtI9YWJA BT 0.42776
i3wAGJaaktw GA 0.36735
eQu1rNs0an0 BT 0.34308
37rzWOQsNIw MS 0.32
vdmoEJ5YbrQ VU 0.31987
WxtbjNsCQ8A BK 0.31279
JgHubY5Vw3Y BT 0.30992
Yi4Ij2NM7U4 MS 0.29664
xwqBXPGE9pQ VU 0.28265
XzYM3PfTM4w VT 0.26705
Bhxk-O1Y7Ho GA 0.26474
0tmA_C6XwfM GA 0.26428
Se3oxnaPsz0 BK 0.2581
E11zDS9XGzg DS 0.23432
qqR6AEXwxoQ BT 0.20889
4wU_LUjG5Ic PR 0.20568
HT5vyqe0Xaw VU 0.19548
akI8YFjEmUw VU 0.15432
J0nA4VgnoCo VT 0.15136
WG0MBPpPC6I MS 0.14965
Hl-__g2gn_A MS 0.14698
b626MiF1ew4 PK 0.12932
98MoyGZKHXc VT 0.10774
jcoYJXDG9sw DS 0.09091
EE-bNr36nyA BK 0.03592
3eYKfiOEJNs GA 0.03293
AwmHb44_ouw VT 0.02863
